# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# Libraries

import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [32]:
# your code here

admissions=pd.read_csv('data/Admission_Predict.csv')

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.

In [33]:
for c in admissions.columns:     
    admissions=admissions.rename(columns={c:c.strip()})

  

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [34]:
# your code here

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [36]:
# your code here

admissions.isnull().sum()[admissions.isnull().sum()>0] 

Series([], dtype: int64)

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [51]:
# your code here

admissions.index=admissions['Serial No.']

admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [60]:
# your code here

admissions.shape==admissions.drop_duplicates(subset=['GRE Score','CGPA']).shape

True

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [66]:
# your code here

admissions.index=[admissions['GRE Score'],admissions['CGPA']]

admissions.head()


,,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
GRE Score,CGPA,,,,,,,,,
337,9.65,1,337,118,4,4.5,4.5,9.65,1,0.92
316,8.00,2,316,104,3,3.0,3.5,8.00,1,0.72
322,8.67,3,322,110,3,3.5,2.5,8.67,1,0.80
314,8.21,4,314,103,2,2.0,3.0,8.21,0,0.65
330,9.34,5,330,115,5,4.5,3.0,9.34,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [81]:
# your code here

len(admissions.loc[(admissions['CGPA']>9)&(admissions['Research'])])

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [86]:
# your code here


admissions.loc[(admissions['CGPA']>9)&(admissions['SOP']<3.5)]

,,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
GRE Score,CGPA,,,,,,,,,
338,9.40,29,338,118,4,3.0,4.5,9.40,1,0.91
327,9.02,63,327,114,3,3.0,3.0,9.02,0,0.61
326,9.11,141,326,114,3,3.0,3.0,9.11,1,0.83
324,9.01,218,324,111,4,3.0,3.0,9.01,1,0.82
325,9.11,382,325,107,3,3.0,3.5,9.11,1,0.84


In [87]:

admissions.loc[(admissions['CGPA']>9)&(admissions['SOP']<3.5)].agg({'Chance of Admit':'mean'})

Chance of Admit    0.802
dtype: float64

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [91]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    return (col-np.mean(col))/np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [98]:
# your code here

admissions['CGPA_std']=standardize(admissions['CGPA'])

admissions['GRE_std']=standardize(admissions['GRE Score'])

admissions['LOR_std']=standardize(admissions['LOR'])

admissions.head()

,,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
GRE Score,CGPA,,,,,,,,,,,,
337,9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
316,8.00,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840
322,8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517
314,8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338
330,9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [99]:
# Libraries
from random import choices

In [102]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [106]:
# your code here


admissions['deciding_column']=admissions.lookup(col_labels=decision_choice, row_labels=admissions.index)

admissions.head()



/var/folders/tx/jymtlyn96zb0qj_hvpjlfm5r0000gn/T/ipykernel_7651/4072980625.py:4: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  admissions['deciding_column']=admissions.lookup(col_labels=decision_choice, row_labels=admissions.index)


,,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
GRE Score,CGPA,,,,,,,,,,,,,
337,9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174
316,8.00,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840
322,8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191
314,8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433
330,9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [112]:
# your code here

def f_decision(row): 
    if row['deciding_column']>0.8:
        return 1
    else:
        return 0

admissions['decision']=admissions.apply(f_decision, axis=1)

admissions.head()

,,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
GRE Score,CGPA,,,,,,,,,,,,,,
337,9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1
316,8.00,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
322,8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0
314,8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
330,9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [120]:
# your code here

admissions['decision'].value_counts()

0    302
1     83
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [146]:
# your code here

admissions.columns=[re.sub('[^\w]','', c.lower()) for c in admissions.columns.str.replace(' ','_')]

admissions.head()

,,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
GRE Score,CGPA,,,,,,,,,,,,,,
337,9.65,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1
316,8.00,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
322,8.67,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0
314,8.21,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
330,9.34,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [166]:
# your code here

def gre_boost(row):
    if row['university_rating']>=4:
        return row['gre_score']+10
    else:
        return row['gre_score']

admissions['adjusted_gre']=admissions.apply(gre_boost, axis=1)

pd.cut(admissions['adjusted_gre'], 4).value_counts()


(305.0, 320.0]     150
(320.0, 335.0]      90
(335.0, 350.0]      75
(289.94, 305.0]     70
Name: adjusted_gre, dtype: int64